In [ ]:
%pip install neuralprophet[live]
%pip install numpy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralprophet import NeuralProphet
#https://github.com/ourownstory/neural_prophet